In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from seai_deap import fab
from seai_deap import dim
from seai_deap import vent

# Load BERPublicsearch data

In [ ]:
berpublicsearch = pd.read_csv("../data/BERPublicsearch.zip")

In [ ]:
berpublicsearch

# Calculate the Heat Loss Parameter for the entire stock

In [ ]:
thermal_bridging = fab.calculate_thermal_bridging(
    wall_area=berpublicsearch["WallArea"],
    roof_area=berpublicsearch["RoofArea"],
    floor_area=berpublicsearch["FloorArea"],
    window_area=berpublicsearch["WindowArea"],
    door_area=berpublicsearch["DoorArea"],
    thermal_bridging_factor=berpublicsearch["ThermalBridgingFactor"],
)

In [ ]:
heat_loss_via_plane_elements = fab.calculate_heat_loss_via_plane_elements(
    wall_area=berpublicsearch["WallArea"],
    roof_area=berpublicsearch["RoofArea"],
    floor_area=berpublicsearch["FloorArea"],
    window_area=berpublicsearch["WindowArea"],
    door_area=berpublicsearch["DoorArea"],
    wall_uvalue=berpublicsearch["UValueWall"],
    roof_uvalue=berpublicsearch["UValueRoof"],
    floor_uvalue=berpublicsearch["UValueFloor"],
    window_uvalue=berpublicsearch["UValueWindow"],
    door_uvalue=berpublicsearch["UvalueDoor"],
)

In [ ]:
fabric_heat_loss = fab.calculate_fabric_heat_loss(
    heat_loss_via_plane_elements=heat_loss_via_plane_elements,
    thermal_bridging=thermal_bridging,
)

In [ ]:
building_volume = dim.calculate_building_volume(
    ground_floor_area=berpublicsearch["GroundFloorArea"],
    first_floor_area=berpublicsearch["FirstFloorArea"],
    second_floor_area=berpublicsearch["SecondFloorArea"],
    third_floor_area=berpublicsearch["ThirdFloorArea"],
    ground_floor_height=berpublicsearch["GroundFloorHeight"],
    first_floor_height=berpublicsearch["FirstFloorHeight"],
    second_floor_height=berpublicsearch["SecondFloorHeight"],
    third_floor_height=berpublicsearch["ThirdFloorHeight"]
)

In [ ]:
ventilation_heat_loss = vent.calculate_ventilation_heat_loss(volume=building_volume)

In [ ]:
heat_loss_coefficient = fab.calculate_heat_loss_coefficient(
    fabric_heat_loss=fabric_heat_loss,
    ventilation_heat_loss=ventilation_heat_loss,
)

In [ ]:
total_floor_area = dim.calculate_total_floor_area(
    ground_floor_area=berpublicsearch["GroundFloorArea"],
    first_floor_area=berpublicsearch["FirstFloorArea"],
    second_floor_area=berpublicsearch["SecondFloorArea"],
    third_floor_area=berpublicsearch["ThirdFloorArea"],
)

In [ ]:
heat_loss_parameter = fab.calculate_heat_loss_parameter(
    heat_loss_coefficient=heat_loss_coefficient,
    total_floor_area=total_floor_area,
)

In [ ]:
heat_loss_parameter

# Extract homes where HLP < 2

According to SEAI any dwelling with a HLP of less than 2 is suitable for a heat pump without a fabric/ventilation upgrade

See SEAI Technical Assessment Process for Heat Pump System Grants: https://www.seai.ie/publications/Technical_Advisor_Role.pdf

In [ ]:
berpublicsearch["heat_loss_parameter"] = heat_loss_parameter

In [ ]:
berpublicsearch_suitable_for_hps = berpublicsearch.query("`heat_loss_parameter` < 2")

In [ ]:
berpublicsearch_suitable_for_hps

In [ ]:
berpublicsearch_not_suitable_for_hps = berpublicsearch.query("`heat_loss_parameter` >= 2")

In [ ]:
berpublicsearch_not_suitable_for_hps

# Plot Year of Construction for suitable homes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ax = sns.countplot(x=berpublicsearch_suitable_for_hps["Year_of_Construction"])

sns.set(rc={'figure.figsize':(40, 40)})
ax.set_xticklabels(ax.get_xticklabels(),rotation=90);